In [4]:
import glob
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import scipy.stats as stats
import utils

# Clean Traffic Data

In [53]:
df['Speed'].dtype == np.float64

True

In [55]:
year = 2015
path = f'../nyc_speed_data/*{year}.csv'

all_files = glob.glob(path)

li = []

for filename in tqdm(all_files):
    df = pd.read_csv(filename, index_col=None, header=0)
    df.columns = ['Id', 'Speed', 'TravelTime', 'Status', 'DataAsOf', 'linkId']
    if df['Speed'].dtype != np.float64:
        cond = ((df['Speed'].str.contains('Speed')) | (df['Speed'].str.contains('Bronx')))
        df[cond]=np.nan
    df['Speed'] = df['Speed'].astype(float)
    df['DataAsOf'] = df['DataAsOf'].astype(str)
    df = df[['Speed','DataAsOf','Id']]
    li.append(df)


# return pd.concat(li, axis=0, ignore_index=True)

100%|██████████| 6/6 [00:08<00:00,  1.46s/it]


In [2]:
year = 2015
df = utils.load_csv(f'../nyc_speed_data/*{year}.csv')

  0%|          | 0/6 [00:00<?, ?it/s]/home/smr1020/anaconda3/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
 50%|█████     | 3/6 [00:02<00:02,  1.47it/s]/home/smr1020/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (0,1,2,3,5) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
 50%|█████     | 3/6 [00:03<00:03,  1.14s/it]


KeyError: 'Speed'

In [ ]:
df = df.dropna()
df['DataAsOf'] = pd.to_datetime(df['DataAsOf'])

In [ ]:
norm_dict = dict{}
df['Speed_norm'] = 0
for group in df.groupby('Id'):
    df_tem = group[1]
    df_tem['Speed_norm'] = (df_tem['Speed'] - df_tem['Speed'].mean())/df_tem.std()
    df['Speed_norm'][df.index.isin(df_tem.index)] = df_tem['Speed_norm']

In [ ]:
df.to_csv('traffic_data_clean.csv')

In [ ]:
df = pd.read_csv('traffic_data_clean.csv')
df['DataAsOf'] = pd.to_datetime(df['DataAsOf'])
df = df[['Id','Speed_norm','DataAsOf']]
df.head()

In [ ]:
start = f'{year}-01-01'
end = f'{year}-12-31'

df_rs = utils.plot_traffic_speed(df,start,end,figsize = (10,3),ticks='15D')

### Remove days where we have less than 80% of sensors working 
If too many sensors are down, the median response won't represent the whole city

In [ ]:
sensor_outage = df_rs.isna().sum(axis=1)
tol = 28

cond_keep = sensor_outage<tol
cond_toss = sensor_outage>tol

speed_ts = df_rs.median(axis=1)

speed_ts[cond_toss] = np.nan

### Visualize removal of days with 20% of sensors malfunctioning

In [ ]:
# plt.figure(figsize = (400,3))
# df_rs.isna().sum(axis=1).plot(xlim=('2019-01-01','2019-12-31'))
# plt.scatter(df_rs.isna().sum(axis=1)[cond_toss].index,
#             df_rs.isna().sum(axis=1)[cond_toss],c='r',zorder=20)

In [ ]:
# plt.figure(figsize = (400,3))
# df_rs.median(axis=1)[cond_keep].plot(xlim=('2019-01-01','2019-12-31'))

# plt.scatter(df_rs.median(axis=1)[cond_toss].index,
#             df_rs.median(axis=1)[cond_toss],c='r')

### A normal day

In [ ]:
speed_ts.groupby(df_rs.index.hour).mean().plot()

# Clean Weather Data

download weather data via api request

In [ ]:
baseurl='https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?'

request = f'station=NYC\
&data=tmpf&data=dwpf&data=p01i&data=wxcodes\
&year1={year}&month1=1&day1=1&year2={year}&month2=12&day2=31&tz=Etc%2FUTC\
&format=onlycomma&latlon=no&missing=M&trace=T&direct=no&report_type=1&report_type=2'

df_weather = pd.read_csv(f'{baseurl}{request}',na_values=['M'])

In [ ]:
# small amounts of precip are marked at T (trace)
df_weather['p01i'][df_weather['p01i'] == 'T'] = 0.001
df_weather['p01i'] = df_weather['p01i'].astype(float)

# set datetime index and make sure only these columns are in the data ['time','tmpf','dwpf','p01i']
df_weather['time'] = pd.to_datetime(df_weather['valid'])
df_weather = df_weather[['time','p01i','wxcodes']]
df_weather['time'] = df_weather['time'] - pd.to_timedelta('1H')
df_weather = df_weather.set_index('time')

#drop na values
df_weather = df_weather.dropna()

# Make snow feature
snow is encoded as a categorical variable (it's hard for these weather stations to measure automatically)

In [ ]:
df_weather['wxcodes'].str.contains('SN').dropna().astype(int).sum()

In [ ]:
df_weather['wxcodes'].str.contains('\-SN').dropna().astype(int).sum()

In [ ]:
df_weather['wxcodes'].str.contains('\+SN').dropna().astype(int).sum()

In [ ]:
((df_weather['wxcodes'].str.contains('\SN')) & ~(df_weather['wxcodes'].str.contains('\-SN')) & ~(df_weather['wxcodes'].str.contains('\+SN'))).sum()

In [ ]:
df_weather = df_weather.dropna()
df_weather['snow'] = 0
df_weather['snow'][df_weather['wxcodes'].str.contains('\-SN')] = 1
df_weather['snow'][(df_weather['wxcodes'].str.contains('\SN'))
                   & ~(df_weather['wxcodes'].str.contains('\-SN'))
                   & ~(df_weather['wxcodes'].str.contains('\+SN'))] = 2
df_weather['snow'][df_weather['wxcodes'].str.contains('\+SN')] = 3

df_weather = df_weather.drop('wxcodes',axis=1)

In [ ]:
df_weather = df_weather.resample('5min').pad()

In [ ]:
df_weather.head()

# Merge Weather and Traffic Data

In [ ]:
df_norm = df_weather
df_norm['speed'] = speed_ts
df_norm = df_norm[df_norm.index.isin(speed_ts.index)]

In [ ]:
means = df_norm.mean()
stds = df_norm.std()
df_norm = (df_norm - means)/stds

# normalize using sin
# df_norm['month'] = np.sin((df_norm.index.month/df_norm.index.month.max())*2*np.pi)
df_norm['weekday'] = (df_norm.index.dayofweek < 5).astype(int)
df_norm['hour'] = np.sin((df_norm.index.hour/df_norm.index.hour.max())*2*np.pi)

df_norm['p01i'] = ((df_weather['p01i'])**(1/3))*2 # cubic normalization for these data
df_norm['snow'] = df_weather['snow'] # no additional norm for these data

In [ ]:
cond = df_norm['speed'].isna()

# df_norm['speed'].interpolate(method='linear',limit=12).plot(c='tab:orange')
# df_norm['speed'].plot(xlim=('2019-07-15 10:00','2019-07-15 14:00'),lw=4,c='tab:blue')

In [ ]:
df_norm = df_norm.interpolate(method='linear',limit=12)
noise = np.random.normal(1,0.08,cond.sum())
df_norm['speed'][cond]=df_norm['speed'][cond]*noise
# df_norm['speed'].plot(xlim=('2019-07-15 10:00','2019-07-15 14:00'))

In [ ]:
week_days = df_norm[df_norm.index.dayofweek < 5]
week_ends = df_norm[df_norm.index.dayofweek > 4]

In [ ]:
normal_wd = week_days.groupby(week_days.index.hour).mean()
normal_we = week_ends.groupby(week_ends.index.hour).mean()
normal_wd['snow'] = 0
normal_wd['p01i'] = 0
normal_we['snow'] = 0
normal_we['p01i'] = 0

In [ ]:
normal_wd.plot()

In [ ]:
normal_we.plot()

In [ ]:
df_copy = df_norm
count = 0

for group in df_copy.groupby(df_copy.index.dayofyear):
    
    n_na = group[1]['speed'].isna().sum().max()
    
    if n_na > 1:
        
        if group[1].index.dayofweek[1] > 4:
        
            copy = group[1].resample('1H').mean()
            copy = normal_we.set_index(copy.index)
            copy = copy.resample('5min').interpolate('cubic')

            noise = np.random.normal(1,0.08,len(copy['speed']))
            copy['speed'] = copy['speed']*noise
            
        if group[1].index.dayofweek[1] < 5:
            
            copy = group[1].resample('1H').mean()
            copy = normal_wd.set_index(copy.index)
            copy = copy.resample('5min').interpolate('cubic')

            noise = np.random.normal(1,0.08,len(copy['speed']))
            copy['speed'] = copy['speed']*noise
            
        df_norm[df_norm.index.isin(copy.index)] = copy
        
        count += 1

In [ ]:
copy.plot()

In [ ]:
group[1].plot()

In [ ]:
df_norm[df_norm.index.dayofyear ==60].isna().sum().max()

In [ ]:
# df_norm['speed'].plot(xlim=('2019-03-01','2019-03-06'))
# # plt.axvline('2019-03-02',c='r')

In [ ]:
# df_norm['speed'].interpolate('linear').plot(xlim=('2019-03-01','2019-03-06'))

In [ ]:
df_norm = df_norm.interpolate('linear')

In [ ]:
df_norm.to_csv(f'df_norm_{year}.csv')

In [ ]:
df_norm.head()